# MLflow

This notebook goes over how to track your LangChain experiments into your MLflow Server

In [4]:
!pip install azureml-mlflow
!pip install pandas
!pip install textstat
!pip install spacy
!pip install openai
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/70.3 kB ? eta -:--:--
     ---------------------------------------- 70.3/70.3 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB 7.9 MB/s eta 0:00:02
     -- ------------------------------------- 0.8/12.8 MB 9.7 MB/s eta 0:00:02
     ---- ----------------------------------- 1.4/12.8 MB 12.9 MB/s eta 0:00:01
     ----- ---------------------------------- 1.9/12.8 MB 12.0 MB/s eta 0:00:01
     ------- -------------------------------- 2.5/12.8 MB 12.4 MB/s eta 0:00:01
     --------- ------------------------------ 3.1/12.8 MB 12.5 MB/s eta 0:00:01
     ----------- ---------------------------- 3.8/12.8 MB 13.4 MB/s eta 0:00:01
     ------------- -------------------------- 4.3/12.8 MB 13.6 MB/s eta 0:00:01
     --------------- ------------------------ 4.9/12.8 MB 13.7 MB/s eta 0:00:01
     ----------------- ---------------------- 5.6/12.8 MB 

In [5]:
import os
os.environ["MLFLOW_TRACKING_URI"] = ""
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["SERPAPI_API_KEY"] = ""

In [6]:
from langchain.callbacks import MLflowCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.llms import OpenAI

NOTE: For beta workflows we have made the default analysis based on textstat and the visualizations based on spacy

In [7]:
"""Main function.

This function is used to try the callback handler.
Scenarios:
1. OpenAI LLM
2. Chain with multiple SubChains on multiple generations
3. Agent with Tools
"""
mlflow_callback = MLflowCallbackHandler()
manager = CallbackManager([mlflow_callback])
llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)

In [ ]:
# SCENARIO 1 - LLM
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
mlflow_callback.flush_tracker(llm, name="simple_sequential")

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
# SCENARIO 2 - Chain
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

test_prompts = [
    {
        "title": "documentary about good video games that push the boundary of game design"
    },
    {"title": "cocaine bear vs heroin wolf"},
    {"title": "the best in class mlops tooling"},
]
synopsis_chain.apply(test_prompts)
mlflow_callback.flush_tracker(synopsis_chain, name="agent")

In [7]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

In [ ]:
# SCENARIO 3 - Agent with Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm, callback_manager=manager)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callback_manager=manager,
    verbose=True,
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)
mlflow_callback.flush_tracker(agent, reset=False, finish=True)